# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 03: Training Data & Feature views</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/3_feature_views_and_training_dataset.ipynb)

<span style="font-width:bold; font-size: 1.4rem;">This notebook explains how to read from a feature group and create training dataset within the feature store</span>

## 🗒️ This notebook is divided into the following sections:

1. Fetch Feature Groups
2. Define Transformation functions
4. Create Feature Views
5. Create Training Dataset with training, validation and test splits

![part2](../../images/02_training-dataset.png) 

## <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [1]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/398
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27;">🪄 Retrieving Feature Groups</span>

In [2]:
application_fg = fs.get_or_create_feature_group(
    name = 'application',
    version = 1
)

bureau_balance_fg = fs.get_or_create_feature_group(
    name = 'bureau_balance',
    version = 1
)

bureau_fg = fs.get_or_create_feature_group(
    name = 'bureau',
    version = 1
)

previous_application_fg = fs.get_or_create_feature_group(
    name = 'previous_application',
    version = 1
)

pos_cash_balance_fg = fs.get_or_create_feature_group(
    name = 'pos_cash_balance',
    version = 1
)

installments_payments_fg = fs.get_or_create_feature_group(
    name = 'installments_payments',
    version = 1
)

credit_card_balance_fg = fs.get_or_create_feature_group(
    name = 'credit_card_balance',
    version = 1
)

previous_loan_counts_fg = fs.get_or_create_feature_group(
    name = 'previous_loan_counts',
    version = 1
)

---

## <span style="color:#ff5f27;">🕵🏻‍♂️ Feature Groups Investigation</span>

We can use `FeatureGroup.show()` method to select top n rows. 

Also we use method `FeatureGroup.read()` in order **to aggregate queries**, which are the output of next methods:

- `FeatureGroup.get_feture()` to get specific feature from our Feature Group.

- `FeatureGroup.select()` to get a few features from our Feature Group.

- `FeatureGroup.select_all()` to get all features from our Feature Group.

- `FeatureGroup.select_except()` to get all features except a few from our Feature Group.

- `FeatureGroup.filter()` in order to apply specific filter to the feature group.

---

## <span style="color:#ff5f27;"> 💼 Feature Selection</span>

In [4]:
fg_query = bureau_fg.select_except(['sk_id_curr','sk_id_bureau'])\
                    .join(application_fg.select_except(['sk_id_curr','flag_mobil',*[f'flag_document_{num}' for num in [2,4,7,10,12,14,17,19,20,21]],'amt_credit', 'weekday_appr_process_start', 'hour_appr_process_start']))\
                    .join(bureau_balance_fg.select_except(['sk_id_bureau','months_balance']))\
                    .join(previous_application_fg.select_except(['sk_id_prev','sk_id_curr','name_contract_type','name_contract_status']))\
                    .join(pos_cash_balance_fg.select_except(['sk_id_prev','sk_id_curr','months_balance', 'name_contract_status', 'sk_dpd', 'sk_dpd_def']))\
                    .join(installments_payments_fg.select_except(['sk_id_prev','sk_id_curr']))\
                    .join(credit_card_balance_fg.select_except(['sk_id_prev','sk_id_curr']))\
                    .join(previous_loan_counts_fg.select_except('sk_id_curr'))

fg_query_show5 = fg_query.show(5)
fg_query_show5

2023-03-02 14:12:58,916 INFO: USE `dowlingj_featurestore`
2023-03-02 14:12:59,372 INFO: SELECT `fg7`.`credit_active` `credit_active`, `fg7`.`credit_currency` `credit_currency`, `fg7`.`days_credit` `days_credit`, `fg7`.`credit_day_overdue` `credit_day_overdue`, `fg7`.`days_credit_enddate` `days_credit_enddate`, `fg7`.`cnt_credit_prolong` `cnt_credit_prolong`, `fg7`.`amt_credit_sum` `amt_credit_sum`, `fg7`.`amt_credit_sum_debt` `amt_credit_sum_debt`, `fg7`.`amt_credit_sum_overdue` `amt_credit_sum_overdue`, `fg7`.`credit_type` `credit_type`, `fg7`.`days_credit_update` `days_credit_update`, `fg0`.`target` `target`, `fg0`.`name_contract_type` `name_contract_type`, `fg0`.`code_gender` `code_gender`, `fg0`.`flag_own_car` `flag_own_car`, `fg0`.`flag_own_realty` `flag_own_realty`, `fg0`.`cnt_children` `cnt_children`, `fg0`.`amt_income_total` `amt_income_total`, `fg0`.`amt_annuity` `amt_annuity`, `fg0`.`amt_goods_price` `amt_goods_price`, `fg0`.`name_type_suite` `name_type_suite`, `fg0`.`name_in

,credit_active,credit_currency,days_credit,credit_day_overdue,days_credit_enddate,cnt_credit_prolong,amt_credit_sum,amt_credit_sum_debt,amt_credit_sum_overdue,credit_type,...,cnt_drawings_atm_current,cnt_drawings_current,cnt_drawings_other_current,cnt_drawings_pos_current,cnt_instalment_mature_cum,name_contract_status,sk_dpd,sk_dpd_def,sk_id_curr,previous_loan_counts
0,Active,currency 1,-622,0,770.0,0,756000.0,674667.0,0.0,Credit card,...,0.0,0,0.0,0.0,36.0,Active,0,0,155227,7
1,Active,currency 1,-622,0,770.0,0,756000.0,674667.0,0.0,Credit card,...,0.0,0,0.0,0.0,36.0,Active,0,0,155227,7
2,Active,currency 1,-622,0,770.0,0,756000.0,674667.0,0.0,Credit card,...,0.0,0,0.0,0.0,36.0,Active,0,0,155227,7
3,Active,currency 1,-622,0,770.0,0,756000.0,674667.0,0.0,Credit card,...,0.0,0,0.0,0.0,36.0,Active,0,0,155227,7
4,Active,currency 1,-622,0,770.0,0,756000.0,674667.0,0.0,Credit card,...,0.0,0,0.0,0.0,36.0,Active,0,0,155227,7


---

## <span style="color:#ff5f27;">🤖 Transformation functions</span>

Hopsworks Feature Store provides functionality to attach transformation functions to training datasets.

Hopsworks Feature Store also comes with built-in transformation functions such as `min_max_scaler`, `standard_scaler`, `robust_scaler` and `label_encoder`.

In [5]:
[t_func.name for t_func in fs.get_transformation_functions()]

['min_max_scaler', 'label_encoder', 'standard_scaler', 'robust_scaler']

We can retrieve transformation function we need .

To attach transformation function to training dataset provide transformation functions as dict, where key is feature name and value is online transformation function name.

Also training dataset must be created from the Query object. Once attached transformation function will be applied on whenever save, insert and get_serving_vector methods are called on training dataset object.

In [6]:
cat_cols = fg_query_show5.dtypes[fg_query_show5.dtypes == 'object'].index
num_cols = fg_query_show5.dtypes[fg_query_show5.dtypes != 'object'].index

mapping_transformer = {col:fs.get_transformation_function(name='label_encoder') for col in cat_cols}
mapping_transformer.update({col:fs.get_transformation_function(name='standard_scaler') for col in num_cols if col !='target'})

---

## <span style="color:#ff5f27;"> ⚙️ Feature View Creation </span>

`Feature Views` stands between **Feature Groups** and **Training Dataset**. Сombining **Feature Groups** we can create **Feature Views** which store a metadata of our data. Having **Feature Views** we can create **Training Dataset**.

The Feature Views allows schema in form of a query with filters, define a model target feature/label and additional transformation functions.

In order to create Feature View we can use `FeatureStore.create_feature_view()` method.

We can specify next parameters:

- `name` - name of a feature group.

- `version` - version of a feature group.

- `labels`- out target variable.

- `transformation_functions` - functions to transform our features.

- `query` - query object with data.

In [7]:
feature_view = fs.create_feature_view(
    name = 'credit_scores',
    version = 1,
    labels = ['target'],
    transformation_functions = mapping_transformer,
    query = fg_query
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/398/fs/335/fv/credit_scores/version/1


For now `Feature View` is saved in Hopsworks and we can retrieve it using `FeatureStore.get_feature_view()`.

In [8]:
feature_view = fs.get_feature_view(
    name = 'credit_scores',
    version = 1
)

---

## <span style="color:#ff5f27;"> 🏋️ Training Dataset Creation</span>

In Hopsworks training data is a query where the projection (set of features) is determined by the parent **FeatureView** with an optional snapshot on disk of the data returned by the query.

**Training Dataset  may contain splits such as:** 
* Training set - the subset of training data used to train a model.
* Validation set - the subset of training data used to evaluate hparams when training a model
* Test set - the holdout subset of training data used to evaluate a mode

Here are some importand things:

- It will inherit the name of FeatureView.

- The feature store currently supports the following data formats for
training datasets: **tfrecord, csv, tsv, parquet, avro, orc**.

- We can choose necessary format using **data_format** parameter.

- **start_time** and **end_time** in order to filter dataset in specific time range.

In [9]:
feature_view.create_train_test_split(
    test_size = 0.2
)

Training dataset job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/398/jobs/named/credit_scores_1_1_create_fv_td_02032023131639/executions


(1, <hsfs.core.job.Job at 0x7f975c8f38b0>)

---

## <span style="color:#ff5f27;">⏭️ **Next:** Part 04 </span>

In the following notebook, we will train a model on the dataset we created in this notebook.